In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR, MSOL
from holodeck import utils, detstats, plot


import hasasia.sensitivity as hsen
import hasasia.sim as hsim

## Goal:
Pass in an hdf5 file from a library of param spaces and return the average detection probability.

## Steps:
1) Read in file: fobs, hc_ss, hc_bg
2) Produce make a PTA set-up calibrated to 50% hc_bg DP?
3) Given pulsar setup and hdf file, calculate average ss DP and average bg DP
    * Calculate bg DP for each of R strain realizations
    * Build S sky realizations
    * Calculate ss DP for each of S sky realizations for each of R strain realizations.


# Step 1. Read in hdf file

In [ ]:
filepath = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/'
filename = filepath+'ss_lib.hdf5'
ssfile = h5py.File(filename, 'r')
print(list(ssfile.keys()))
# print(ssfile['pta_dur'][0])
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
dur = ssfile['pta_dur'][0]
cad = ssfile['pta_cad'][0]
hc_ss_all = ssfile['hc_ss'][...]
hc_bg_all = ssfile['hc_bg'][...]
shape = hc_ss_all.shape
nsamp, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print(nsamp, nfreqs, nreals, nloudest)
ssfile.close()

In [ ]:
print('samples (N):', nsamp, '\nreals (R):', nreals, '\nN,F,R,L:', hc_ss_all.shape)

## single realization example

# Step 2. build PTA
Here I am doing this randomly, but really it should be calibrated to hc_bg. Unsure how to do this, if it should be calibrated independently for each strain realization (R), a single pta should be used for all calculations, or positions etc. should be randomized for each iteration. We could even have pta realizations, on top of the strain realizations and the (single source) sky realizations, but that sounds like a nightmare.

The pulsars object is used to store thetas, phis, and sigmas of the pta. With this, use the detect_bg_pta(pulsars, spectra,...) and detect_ss_pta(pulsars,...) functions in detstats. Alternatively, you can just pass in the phis, thetas, and sigmas array to the detect_bg() and detect_ss() functions.

In [ ]:
npsrs = 30
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
sigmas = np.ones_like(phis)*1.17e-5
# build sim_pta
psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                    phi=phis, theta=thetas)


calibrate to middle-ish sample

In [ ]:
dp_bg = detstats.detect_bg_pta(psrs, fobs, cad, 
                               hc_bg_all[int(nsamp/2)])
print(np.mean(dp_bg))

# Step 3. Calculate detstats
Given pulsar setup, and hdf file, calculate average ss DP

Parameters and functions needed:
* def detect_bg_pta(
    - pulsars, 
    - fobs, 
    - cad, 
    - hc_bg, 
    - alpha_0=0.001, 
    - ret_all = False):
* def detect_ss_pta(
    - pulsars, 
    - cad, 
    - dur, 
    - fobs,
    - dfobs, 
    - hc_ss, 
    - hc_bg
    - theta_ss
    - phi_ss=None, 
    - Phi0_ss=None, 
    - iota_ss=None, 
    - psi_ss=None):

## 3.1 BG DP

In [ ]:
dp_bg_all = np.zeros((nsamp, nreals)) # (N,R)
for nn in range(nsamp):
    dp_bg_all[nn,:] = detstats.detect_bg_pta(psrs, fobs, cad, hc_bg_all[nn])


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.set_xlabel('Param Space Sample')
ax.set_ylabel('Detection Probability, $\gamma$')
ax.errorbar(np.arange(nsamp), np.mean(dp_bg_all, axis=1), 
            yerr = np.std(dp_bg_all, axis=1), linestyle='', 
            marker='o', capsize=5, color='cornflowerblue', 
            label = r'$\langle \gamma_\mathrm{BG} \rangle$')
ax.set_yscale('log')
fig.legend()

## 3.2 SS DP

Build Sky Realizations

In [ ]:
nskies = 25 # choose the number of ss sky realizations
theta_ss = np.random.uniform(0, np.pi, size = nfreqs * nskies * nloudest).reshape(nfreqs, nskies, nloudest)
phi_ss = np.random.uniform(0, 2*np.pi, size = theta_ss.size).reshape(theta_ss.shape)
Phi0_ss = np.random.uniform(0,2*np.pi, size = theta_ss.size).reshape(theta_ss.shape)
iota_ss = np.random.uniform(0,  np.pi, size = theta_ss.size).reshape(theta_ss.shape)
psi_ss = np.random.uniform(0,   np.pi, size = theta_ss.size).reshape(theta_ss.shape)

Calculate ss DP

In [ ]:
dp_ss_all = np.zeros((nsamp, nreals, nskies)) # (N,R,S)
for nn in range(nreals):
    dp_ss_all[nn,:,:] = detstats.detect_ss_pta(psrs, cad, dur, fobs, dfobs,
                                               hc_ss_all[nn], hc_bg_all[nn],
                                               theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss,
                                               iota_ss=iota_ss, psi_ss=psi_ss)

In [ ]:
def plot_detprob(dp_ss_all, dp_bg_all):
    """ Plot detection probability for many samples.

    Paramaters
    ----------
    dp_ss_all : (N,R) NDarray
    dp_bg_all : (N,R,S) NDarray
    """
    fig, ax = plt.subplots(figsize=(6.5,4))
    ax.set_xlabel('Param Space Sample')
    ax.set_ylabel('Detection Probability, $\gamma$')
    ax.errorbar(np.arange(nsamp), np.mean(dp_bg_all, axis=1), 
                yerr = np.std(dp_bg_all, axis=1), linestyle='', 
                marker='d', capsize=5, color='cornflowerblue', alpha=0.75,
                label = r'$\langle \gamma_\mathrm{BG} \rangle$')
    ax.errorbar(np.arange(nsamp), np.mean(dp_ss_all, axis=(1,2)),
                yerr = np.std(dp_ss_all, axis=(1,2)), linestyle='', 
                marker='o', capsize=5, color='orangered', alpha=0.75,
                label = r'$\langle \gamma_\mathrm{SS} \rangle$')
    ax.set_yscale('log')
    ax.set_title('Average DP across Realizations')

    ax.legend()
    fig.tight_layout()

    return fig

fig = plot_detprob(dp_ss_all, dp_bg_all)

In [ ]:
# plot different sky realizations separately
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('Param Space Sample')
ax.set_ylabel('Detection Probability, $\gamma$')
ax.errorbar(np.arange(nsamp), np.mean(dp_bg_all, axis=1), 
            yerr = np.std(dp_bg_all, axis=1), linestyle='', 
            marker='d', capsize=5, color='cornflowerblue', 
            label = r'$\langle \gamma_\mathrm{BG} \rangle$')
for ss in range(nskies): # each color is a sky realization
    if ss == 0: ss_label = r'$\langle \gamma_\mathrm{SS} \rangle$'
    else: ss_label = None
    ax.errorbar(np.arange(nsamp), np.mean(dp_ss_all[:,:,ss], axis=1),
                yerr = np.std(dp_ss_all[:,:,ss], axis=1), linestyle='', 
            marker='o', capsize=5, alpha=0.25,
            label = ss_label)
ax.set_yscale('log')
ax.legend()

## Count realizations with detections

In [ ]:
'''
def detfrac_from_reals(dp_ss_all, dp_bg_all, thresh=0.5):
    """ 
    Calculate how many realizations have a detections, to get 
    the detection probability for a sample.

    Parameters
    ----------
    dp_ss_all : (N, R, S) NDarray
    dp_bg_all : (N, R) NDarray
    thresh : scalar
        threshold DP to count as a detection

    Returns
    -------
    ss_detfrac : (N,) NDarray
        Fraction of realizations with an ss DP above thresh.
    bg_detfrac : (N,) NDarray
        Fraction of realizations with a bg DP above thresh.
    
    """
    nsamp = len(dp_ss_all)
    nreals = len(dp_ss_all[0])
    nskies = len(dp_ss_all[0,0])
    bg_detfrac = np.zeros(nsamp)
    ss_detfrac = np.zeros(nsamp)
    for nn in range(nsamp):
        count_bg = 0
        count_ss = 0
        for rr in range(nreals):
            if (dp_bg_all[nn,rr] > thresh):
                count_bg += 1
            for ss in range(nskies):
                if (dp_ss_all[nn,rr,ss] > thresh):
                    count_ss += 1
        bg_detfrac[nn] = count_bg/nreals
        ss_detfrac[nn] = count_ss/(nreals*nskies)

    return ss_detfrac, bg_detfrac

'''

def detfrac_from_reals(dp_ss_all, dp_bg_all, thresh=0.5):
    """ 
    Calculate how many realizations have a detections, to get 
    the detection probability for a sample.

    Parameters
    ----------
    dp_ss_all : (N, R, S) NDarray
    dp_bg_all : (N, R) NDarray
    thresh : scalar
        threshold DP to count as a detection

    Returns
    -------
    ss_detfrac : (N,) NDarray
        Fraction of realizations with an ss DP above thresh.
    bg_detfrac : (N,) NDarray
        Fraction of realizations with a bg DP above thresh.
    
    """
    nsamp = len(dp_ss_all)
    nreals = len(dp_ss_all[0])
    nskies = len(dp_ss_all[0,0])
    bg_detfrac = np.zeros(nsamp)
    ss_detfrac = np.zeros(nsamp)
    for nn in range(nsamp):
        count_bg = 0
        count_ss = 0
        for rr in range(nreals):
            if (dp_bg_all[nn,rr] > thresh):
                count_bg += 1
            for ss in range(nskies):
                if (dp_ss_all[nn,rr,ss] > thresh):
                    count_ss += 1
        bg_detfrac[nn] = count_bg/nreals
        ss_detfrac[nn] = count_ss/(nreals*nskies)

    return ss_detfrac, bg_detfrac

In [ ]:
thresh=0.5
ss_detfrac, bg_detfrac = detfrac_from_reals(dp_ss_all, dp_bg_all, thresh)
def plot_detfrac(df_ss, df_bg, thresh):
    fig, ax = plt.subplots(figsize=(6.5,4))
    ax.plot(np.arange(nsamp), df_bg, color='cornflowerblue', label='BG',
            marker='d')
    ax.plot(np.arange(nsamp), df_ss, color='orangered', label='SS',
            marker='o')
    ax.set_xlabel('Param Space Sample')
    ax.set_ylabel('Detection Fraction')
    ax.set_title('Fraction of Realizations with DP > %0.2f' % thresh)
    ax.legend()
    fig.tight_layout()
    return fig
fig = plot_detfrac(ss_detfrac, bg_detfrac, thresh)

# Streamline
 Do it all for a given pta

In [ ]:
def run_psr_calculations(hc_ss_all, hc_bg_all, npsrs = 30, sigma = 1.17e-5, 
                         nskies = 25):
    shape = hc_ss_all.shape
    nsamp, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]

    # build PTA
    phis = np.random.uniform(0, 2*np.pi, size = npsrs)
    thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
    sigmas = np.ones_like(phis)*sigma
    psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                    phi=phis, theta=thetas)
    
    # Calculate background DP
    dp_bg_all = np.zeros((nsamp, nreals)) # (N,R)
    for nn in range(nsamp):
        dp_bg_all[nn,:] = detstats.detect_bg_pta(psrs, fobs, cad, hc_bg_all[nn])

    # Build ss skies
    theta_ss = np.random.uniform(0, np.pi, size = nfreqs * nskies * nloudest).reshape(nfreqs, nskies, nloudest)
    phi_ss = np.random.uniform(0, 2*np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    Phi0_ss = np.random.uniform(0,2*np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    iota_ss = np.random.uniform(0,  np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    psi_ss = np.random.uniform(0,   np.pi, size = theta_ss.size).reshape(theta_ss.shape)

    # Calculate single source DP
    dp_ss_all = np.zeros((nsamp, nreals, nskies)) # (N,R,S)
    for nn in range(nreals):
        dp_ss_all[nn,:,:] = detstats.detect_ss_pta(psrs, cad, dur, fobs, dfobs,
                                                hc_ss_all[nn], hc_bg_all[nn],
                                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss,
                                                iota_ss=iota_ss, psi_ss=psi_ss)
    return dp_ss_all, dp_bg_all
        

In [ ]:
dp_ss_psr, dp_bg_psr = run_psr_calculations(hc_ss_all, hc_bg_all, npsrs = 30, sigma = 1e-6,
                                  nskies = 25)
fig1 = plot_detprob(dp_ss_psr, dp_bg_psr)

In [ ]:
print(np.min(dp_ss_psr))

where are the rest of my single sources? possibly too low to detect

In [ ]:
thresh=0.5
df_ss_psr, df_bg_psr = detfrac_from_reals(dp_ss_psr, dp_bg_psr, thresh)
fig2 = plot_detfrac(df_ss_psr, df_bg_psr, thresh)

Try another pta, 40 pulsars, 5e-7 sigma

In [ ]:
dp_ss_psr40, dp_bg_psr40 = run_psr_calculations(hc_ss_all, hc_bg_all, npsrs = 40, sigma = 5e-7,
                                  nskies = 25)
fig1 = plot_detprob(dp_ss_psr40, dp_bg_psr40)
thresh=0.5
df_ss_psr40, df_bg_psr40 = detfrac_from_reals(dp_ss_psr40, dp_bg_psr40, thresh)
fig2 = plot_detfrac(df_ss_psr40, df_bg_psr40, thresh)

In [ ]:
def plot_nsample_dp(dp_ss, dp_bg, nn):
    fig, ax = plt.subplots(figsize=(6.5,4))
    for ss in range(nskies):
        ax.scatter(np.arange(nreals), dp_ss_psr40[nn,:,ss], alpha=0.5)
    ax.scatter(np.arange(nreals), dp_bg_psr40[nn], color='k')
    ax.set_xlabel('Realization (R)')
    ax.set_ylabel('SS DetProb')
    ax.set_title('Sample (N): %d' %nn)
    return fig
    

fig = plot_nsample_dp(dp_ss_psr40, dp_bg_psr40, nn=7)
fig = plot.plot_bg_ss(fobs, hc_bg_all[nn], hc_ss_all[nn])

In [ ]:
fig1 = plot_nsample_dp(dp_ss_psr40, dp_bg_psr40, nn=20)
# ax = fig1.axes[0]
# ax.set_yscale('log') # 0's dont show up on log plot
# ax.set_ylim(0, 1e-40)
fig2 = plot.plot_bg_ss(fobs, hc_bg_all[20], hc_ss_all[20])
# It really just depends on how high the strain is!


In [ ]:
def plot_sample_nn(fobs, hc_ss, hc_bg, dp_ss, dp_bg, df_ss, df_bg, nn):
    """ Plot strain and detection probability for a single sample.
    """
    shape = hc_ss.shape
    F, R, L = shape[0], shape[1], shape[2]
    S = dp_ss.shape[-1]
    fig, axs = plt.subplots(1, 2,figsize=(12,4))

    # Strains
    plot.draw_ss_and_gwb(axs[0], fobs*YR, hc_ss, hc_bg)
    axs[0].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
    axs[0].set_ylabel(plot.LABEL_CHARACTERISTIC_STRAIN)
    axs[0].set_title('Sample nn=%d (F=%d, R=%d, L=%d)' % (nn, F, R, L))
    axs[0].set_xscale('log')
    axs[0].set_yscale('log')

    for ss in range(S):
        axs[1].scatter(np.arange(nreals), dp_ss[:,ss], alpha=0.25)
    axs[1].scatter(np.arange(nreals), dp_bg, color='k', 
                   label='BG, DF = %.2e' % df_bg,
                   marker='d')
    axs[1].errorbar(np.arange(nreals), np.mean(dp_ss[:,:], axis=1),
                     yerr = np.std(dp_ss[:,:], axis=1), color='orangered',
                      label = 'SS, sky-avg, DF = %.2e' % df_ss, linestyle='', capsize=3,
                       marker='o' )
    axs[1].set_xlabel('Realization (R)')
    axs[1].set_ylabel('SS DetProb')
    # axs[1].set_title('BG DF = ' %nn)
    fig.legend()
    fig.tight_layout()
    
    return fig
nn=7
fig = plot_sample_nn(fobs, hc_ss_all[nn], hc_bg_all[nn], 
                     dp_ss_psr40[nn], dp_bg_psr40[nn],
                     df_ss_psr40[nn], df_bg_psr40[nn], 
                     nn)

In [ ]:
for nn in range(nsamp):
    fig = plot_sample_nn(fobs, hc_ss_all[nn], hc_bg_all[nn],
                         dp_ss_psr40[nn], dp_bg_psr40[nn], 
                         df_ss_psr40[nn], df_bg_psr40[nn], nn=nn)
    fname = (filepath+'detstats/p%06d_detprob.png' % nn)
    print(fname)
    fig.savefig(fname, dpi=100)
    plt.close(fig)

# Appendix: Regular Old Detection Statistic Calculation

### A.1 build sam and calculate gws

In [ ]:
dur = 15.0*YR
cad = .2*YR
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
dfobs = np.diff(fobs_edges)
sam = holo.sam.Semi_Analytic_Model() 
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
hc_ss_one, hc_bg_one = sam.ss_gwb(fobs_edges, realize=20, loudest=5)

### A.2 build pta

In [ ]:
npsrs = 40
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
sigmas = np.ones_like(phis)*5.2e-6
# build sim_pta
pulsars = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                    phi=phis, theta=thetas)


### A.3 calculate bg DP

In [ ]:
dp_bg1 = detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg_one)
dp_bg2 = detstats.detect_bg(thetas, phis, sigmas, fobs, cad, hc_bg_one)
plt.scatter(dp_bg1, dp_bg2)
print(dp_bg1)
print(dp_bg2)
plt.xlabel('detect_bg_pta')
plt.ylabel('detect_bg')
plt.title(('F,R,L ='+ str(hc_ss_one.shape)+'\nsigmas=%.2e' %sigmas[0]))


### A.4 build ss sky realizations

In [ ]:
print(hc_ss_one.shape)

In [ ]:
S = 15
F = len(fobs)
L = len(hc_ss_one[0,0])
theta_ss = np.random.uniform(0, np.pi, size=F*S*L).reshape(F,S,L)
phi_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
Phi0_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
iota_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)
psi_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)

### A.5 calculate ss DP

In [ ]:
dp_ss1 = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss_one, hc_bg_one,
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss)
dp_ss2 = detstats.detect_ss(thetas, phis, sigmas, cad, dur, fobs, dfobs, hc_ss_one, hc_bg_one,
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss)

In [ ]:
plt.scatter(dp_ss1, dp_ss2)
print(dp_ss1.shape, utils.stats(dp_ss1))
print(dp_ss2.shape, utils.stats(dp_ss2))
plt.xlabel('detect_ss_pta')
plt.ylabel('detect_ss')
plt.title(('F,R,L ='+ str(hc_ss_one.shape)+'\nsigmas=%.2e' %sigmas[0]))


In [ ]:
print(dp_ss1.shape) # (R, S)

In [ ]:
def plot_dpbg_dpss(dpbg, dpss, title):
    R = len(dpbg)
    fig, ax = plt.subplots(figsize = (8,4))
    ax.scatter(np.arange(R), dpbg, label='$\gamma_\mathrm{bg}$, avg = %.2e' % np.mean(dpbg), color='blue')
    ax.errorbar(np.arange(R), np.mean(dpss, axis=1), yerr=np.std(dpss, axis=1), capsize=3,
                linestyle='', marker='o',
                color='red', label = '$\gamma_\mathrm{SS}$ (averaged over sky realizations), avg = %.2e' 
                % np.mean(dpss))
    for ss in range(len(dpss[0])):
        ax.scatter(np.arange(R), dpss[:,ss], alpha=0.25)

    fig.legend(bbox_to_anchor=(0.1,0), bbox_transform=fig.transFigure, loc='upper left')
    ax.set_xlabel('Realization')
    ax.set_ylabel('Detection Probability')
    ax.set_yscale('log')
    ax.set_title(title)
    # plt.ylim(0,1)
    return fig

fig = plot_dpbg_dpss(dp_bg1, dp_ss1, title = ('F,R,L ='+ str(hc_ss_one.shape)+'\nsigmas=%.2e' %sigmas[0]))

### A.6 %timeit

In [ ]:
%timeit detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss_one, hc_bg_one,\
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss) 

In [ ]:
%timeit detstats.detect_ss(thetas, phis, sigmas, cad, dur, fobs, dfobs, hc_ss_one, hc_bg_one, \
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss)

In [ ]:
%timeit detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg_one)
%timeit detstats.detect_bg(thetas, phis, sigmas, fobs, cad, hc_bg_one)


## A.7 Vary L
### L = 100

In [ ]:
def calculate_dps(pulsars, hc_ss, hc_bg, S=15):
    F = len(hc_ss)
    R = len(hc_ss[0])
    L = len(hc_ss[0,0])

    # build ss sky realizations        
    theta_ss = np.random.uniform(0, np.pi, size=F*S*L).reshape(F,S,L)
    phi_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    Phi0_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    iota_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    psi_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)

    dp_bg = detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg)
    dp_ss = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg,
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, 
                                iota_ss=iota_ss, psi_ss=psi_ss)
    return dp_bg, dp_ss 

def time_dps(pulsars, hc_ss, hc_bg, S=15):
    F = len(hc_ss)
    R = len(hc_ss[0])
    L = len(hc_ss[0,0])

    # build ss sky realizations        
    theta_ss = np.random.uniform(0, np.pi, size=F*S*L).reshape(F,S,L)
    phi_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    Phi0_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    iota_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    psi_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)

    %timeit detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg)
    %timeit detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg,\
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, \
                                iota_ss=iota_ss, psi_ss=psi_ss)


In [ ]:
hc_ss_L100, hc_bg_L100 = sam.ss_gwb(fobs_edges, realize=20, loudest=100)
dp_bg_L100, dp_ss_L100 = calculate_dps(pulsars, hc_ss_L100, hc_bg_L100, S=15)

In [ ]:
fig = plot_dpbg_dpss(dp_bg_L100, dp_ss_L100, 
                     title = ('F,R,L ='+ str(hc_ss_L100.shape)+'\nsigmas=%.2e' %sigmas[0]))

### L = 1

In [ ]:
hc_ss_L1, hc_bg_L1 = sam.ss_gwb(fobs_edges, realize=20, loudest=1)
dp_bg_L1, dp_ss_L1 = calculate_dps(pulsars, hc_ss_L1, hc_bg_L1, S=15)
fig = plot_dpbg_dpss(dp_bg_L1, dp_ss_L1, 
                     title = ('F,R,L ='+ str(hc_ss_L1.shape)+'\nsigmas=%.2e' %sigmas[0]))

## A.8 % Timeit 

In [ ]:
time_dps(pulsars, hc_ss_L1, hc_bg_L1, S=15)

In [ ]:
time_dps(pulsars, hc_ss_L100, hc_bg_L100, S=15)